In [ ]:
import torch                                            # Main deep learning library.
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms            # Handles datasets and transfomations.
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter       # Track training loss

/home/chris/venv/cdd/lib/python3.12/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(
2025-03-11 15:25:19.215112: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-11 15:25:19.232877: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-11 15:25:19.237925: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 15:25:19.251331: I tensorflow/co

In [ ]:
# Define Variational Autoencoder (VAE) Model 

### See Conditional VAE that introduces and additional conditioning variable, tipycally lables, allowing for controled generation.

'''
This class defines the VAE model. It consists of an encoder and a decoder. 
The encoder compresses the input image into a latent space, and the decoder reconstructs 
the image from the latent space.
'''

class VAE(nn.Module):
    def __init__(self, latent_dim=128):                             # latent_dim=128: The size of latent representation.
        super(VAE, self).__init__()
        self.latent_dim = latent_dim

        # Encoder                                                   # Features extraction: 4 concolutional layers extract hierarchical image features,
        self.encoder = nn.Sequential(                               # each layer halves the images size while increasing feature maps.
            nn.Conv2d(3, 32, 4, 2, 1), nn.ReLU(),
            nn.Conv2d(32, 64, 4, 2, 1), nn.ReLU(),
            nn.Conv2d(64, 128, 4, 2, 1), nn.ReLU(),
            nn.Conv2d(128, 256, 4, 2, 1), nn.ReLU()
        )
        self.fc_mu = nn.Linear(256 * 16 * 16, latent_dim)           # Latent Space Mwean and Variance: fc_mu and fc_logvar predict mean and          
        self.fc_logvar = nn.Linear(256 * 16 * 16, latent_dim)       # log-variance for the latent distribution.

        # Decoder                                                   # Image reconstruction: Fulli connected layer (fc_dec) expands latent vector.
        self.fc_dec = nn.Linear(latent_dim, 256 * 16 * 16)          # Tranpose convolutions upsample back to 256x256 image.
        self.decoder = nn.Sequential(                               # Final layer uses Sigmoid() to constrain pixel values to [0, 1].
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1), nn.ReLU(),
            nn.ConvTranspose2d(32, 3, 4, 2, 1), nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):                           # Reparametrization trick, allows backpropagation the through the 
        std = torch.exp(0.5 * logvar)                               # stocastic sampling process. Converts mean and log-variance into a latent vactor.
        eps = torch.randn_like(std)                                 # Allows backpropagation through a stochastic operation.
        return mu + eps * std

    def forward(self, x):                                           # Forward pass of the VAE. It encodes the input, samples from latent space, and 
        x = self.encoder(x).view(x.size(0), -1)                     # decodes the sample back to the image space. 1) Encodes image --> latent vector(mu, logvar).
        mu, logvar = self.fc_mu(x), self.fc_logvar(x)               # 2) Samples z using reparametrization trick. 3) Decodes z baxk to an image.
        z = self.reparameterize(mu, logvar)
        x = self.fc_dec(z).view(x.size(0), 256, 16, 16)
        x = self.decoder(x)
        return x, mu, logvar

In [ ]:
# Training Function
'''
This function trains the VAE. It initializes the model, optimizer, and TensorBoard writer. 
It then iterates over the dataset for a specified number of epochs, computes the loss, performs 
backpropagation, and updates the model parameters. The loss is logged to TensorBoard, 
and the trained model is saved.
'''

def train_vae(epochs=10, batch_size=32, latent_dim=128):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")           # Uses GPU if avalaible.
    dataloader = get_dataloader(batch_size)                                         # Load dataset using DataLoader.
    vae = VAE(latent_dim).to(device)                                                # Initializes model and Adam optimizer.
    optimizer = optim.Adam(vae.parameters(), lr=1e-4)                               # Creates TensorBoard logger.
    writer = SummaryWriter("runs/vae_experiment")                                   

    for epoch in range(epochs):
        total_loss = 0
        for images, _ in dataloader:
            images = images.to(device)
            optimizer.zero_grad()
            x_recon, mu, logvar = vae(images)
            loss = vae_loss(x_recon, images, mu, logvar)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader.dataset)                             # Log loss and save model. Save loss TensorBoard and save trained model for later use.
        writer.add_scalar('Loss/train', avg_loss, epoch)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")
    
    writer.close()
    torch.save(vae.state_dict(), "vae_256x256.pth")
    return vae

In [ ]:
# Generate and Visualize Samples

'''
This function generates and visualizes images from the trained VAE. 
It samples random latent vectors, decodes them into images, and 
displays the images using Matplotlib.
'''

def generate_images(vae, num_images=5, latent_dim=128):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    vae.to(device)
    vae.eval()
    with torch.no_grad():
        z = torch.randn(num_images, latent_dim).to(device)                                  # Samples random latent vectors (z).
        generated_images = vae.decoder(vae.fc_dec(z).view(num_images, 256, 16, 16))         # Uses decoder to generate images from z.
    
    generated_images = generated_images.cpu().numpy().transpose(0, 2, 3, 1)                 # Converts tensor tu NumPy forma for visualization.
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))                               
    for i, ax in enumerate(axes):
        ax.imshow(generated_images[i])
        ax.axis("off")
    plt.show()

# Training Example
# vae = train_vae(epochs=10)
# generate_images(vae)


In [ ]:
### Flax JAX implementation

import jax
import jax.numpy as jnp
import flax.linen as nn
import optax

class VAE(nn.Module):
    latent_dim: int

    def setup(self):
        self.encoder = nn.Sequential([
            nn.Conv(32, (4, 4), strides=2), nn.relu,
            nn.Conv(64, (4, 4), strides=2), nn.relu,
            nn.Conv(128, (4, 4), strides=2), nn.relu,
            nn.Conv(256, (4, 4), strides=2), nn.relu,
        ])
        self.fc_mu = nn.Dense(self.latent_dim)
        self.fc_logvar = nn.Dense(self.latent_dim)
        self.fc_dec = nn.Dense(256 * 16 * 16)
        self.decoder = nn.Sequential([
            nn.ConvTranspose(128, (4, 4), strides=2), nn.relu,
            nn.ConvTranspose(64, (4, 4), strides=2), nn.relu,
            nn.ConvTranspose(32, (4, 4), strides=2), nn.relu,
            nn.ConvTranspose(3, (4, 4), strides=2), nn.sigmoid,
        ])

    def __call__(self, x):
        x = self.encoder(x)
        x = x.reshape((x.shape[0], -1))
        mu, logvar = self.fc_mu(x), self.fc_logvar(x)
        std = jnp.exp(0.5 * logvar)
        eps = jax.random.normal(jax.random.PRNGKey(0), std.shape)
        z = mu + eps * std
        x = self.fc_dec(z).reshape((-1, 256, 16, 16))
        return self.decoder(x), mu, logvar



### TensorFlow (Keras) implementation

import tensorflow as tf
from tensorflow.keras import layers

class VAE(tf.keras.Model):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Conv2D(32, (4, 4), strides=2, activation='relu'),
            layers.Conv2D(64, (4, 4), strides=2, activation='relu'),
            layers.Conv2D(128, (4, 4), strides=2, activation='relu'),
            layers.Conv2D(256, (4, 4), strides=2, activation='relu'),
            layers.Flatten()
        ])
        self.fc_mu = layers.Dense(latent_dim)
        self.fc_logvar = layers.Dense(latent_dim)
        self.fc_dec = layers.Dense(256 * 16 * 16)
        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(128, (4, 4), strides=2, activation='relu'),
            layers.Conv2DTranspose(64, (4, 4), strides=2, activation='relu'),
            layers.Conv2DTranspose(32, (4, 4), strides=2, activation='relu'),
            layers.Conv2DTranspose(3, (4, 4), strides=2, activation='sigmoid'),
        ])

    def call(self, x):
        x = self.encoder(x)
        mu, logvar = self.fc_mu(x), self.fc_logvar(x)
        std = tf.exp(0.5 * logvar)
        eps = tf.random.normal(std.shape)
        z = mu + eps * std
        x = tf.reshape(self.fc_dec(z), (-1, 256, 16, 16))
        return self.decoder(x), mu, logvar
